<a href="https://colab.research.google.com/github/ZubekKlaudia/ZastosowanieUczeniaMaszynowego/blob/main/Zastosowanie_g%C5%82%C4%99bokiego_uczenia_maszynowego_do_rozpoznawania_ras_ps%C3%B3w_poprawki_14_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#„Zastosowanie głębokiego uczenia maszynowego do rozpoznawania ras psów na podstawie zdjęć”

Autor: Klaudia Zubek

Import niezbędnych bibliotek

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from PIL import Image
import pandas as pd
from collections import Counter
import random
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
import joblib
import seaborn as sns
from IPython.display import display
import tensorflow as tf
from tensorflow.keras.preprocessing import image

#Import zbioru danych Stanford Dog Datasets

In [ ]:
# Instalacja biblioteki Kaggle, pobranie i rozpakowanie zbioru danych Stanford Dogs
!pip install -q kaggle
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset --unzip
data_dir = './'
image_directory = os.path.join(data_dir, './images/Images/')

#Eksploracja Danych (EDA - Exploratory Data Analysis)

In [ ]:
# Policzenie klas oraz prezentacja przykładowych wyników
classes = os.listdir(image_directory)

print(f"Liczba klas (ras psów): {len(classes)}")
print("Przykładowe klasy:")
print(classes[:5])


In [ ]:
# Policzenie przykładów w każdej klasie i wyświetlenie posortowanej tabeli
class_counts = {folder: len(os.listdir(os.path.join('./images/Images/', folder))) for folder in os.listdir('./images/Images/')}

class_counts_df = pd.DataFrame.from_dict(class_counts, orient='index', columns=['Liczba przykładów'])
class_counts_df.index.name = 'Klasa'
class_counts_df.reset_index(inplace=True)

class_counts_df_sorted = class_counts_df.sort_values(by='Liczba przykładów', ascending=False)

display(class_counts_df_sorted)


In [ ]:
# Obliczenie zakresu liczebności klas i wyświetlenie różnicę między największą a najmniejszą wartością
max_count = class_counts_df_sorted['Liczba przykładów'].max()
min_count = class_counts_df_sorted['Liczba przykładów'].min()
range_count = max_count - min_count

print(f"Największa liczba przykładów: {max_count}")
print(f"Najmniejsza liczba przykładów: {min_count}")
print(f"Zakres liczebności klas (różnica): {range_count}")


#Wizualizacja obrazów

In [ ]:
# Wyświetlenie obrazów z przykładowej klasy
example_class = os.listdir(image_directory)[0]
example_images = os.listdir(os.path.join(image_directory, example_class))[:5]

plt.figure(figsize=(15, 5))
for i, img_name in enumerate(example_images):
    img_path = os.path.join(image_directory, example_class, img_name)
    img = Image.open(img_path)
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(example_class,fontsize=7)
    plt.axis('off')
plt.show()

In [ ]:
# Wyświetlenie obrazów z przykładowych klas
plt.figure(figsize=(10, 10))
for i, cls in enumerate(classes[:9]):
    img_name = random.choice(os.listdir(os.path.join(image_directory, cls)))
    img_path = os.path.join(image_directory, cls, img_name)
    img = Image.open(img_path)
    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.title(cls, fontsize=8)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Wyświetlenie liczby uszkodzonych obrazów
damaged_images = []
for cls in classes:
    for img_name in os.listdir(os.path.join(image_directory, cls)):
        img_path = os.path.join(image_directory, cls, img_name)
        try:
            img = Image.open(img_path)
            img.verify()
        except:
            damaged_images.append(img_path)

print(f"Liczba uszkodzonych obrazów: {len(damaged_images)}")
if damaged_images:
    print("Przykładowe uszkodzone obrazy:", damaged_images[:5])

#Przygotowanie danych do uczenia

In [ ]:
# Ustawienie ścieżek do danych
train_data_folder = './data/train'
validation_data_folder = './data/val'
test_data_folder = './data/test'

In [ ]:
# Utwórzenie katalogów i podział danych w odpowiedniej proporcji na zbior treningowy, walidacyjny i testowy
os.makedirs(train_data_folder, exist_ok=True)
os.makedirs(validation_data_folder, exist_ok=True)
os.makedirs(test_data_folder, exist_ok=True)

classes = os.listdir(image_directory)

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

#Pętla for iteruje przez wszystkie klasy
for class_name in classes:
    if class_name.startswith('.'):
        continue
    class_path = os.path.join(image_directory, class_name)
    if not os.path.isdir(class_path):
        continue

    #Następuje pobranie wszystkich obrazów w folderze danej klasy
    images = os.listdir(class_path)

    #Podział danych
    train_images, val_test_images = train_test_split(images, test_size=(1 - train_ratio))
    val_images, test_images = train_test_split(val_test_images, test_size=(test_ratio / (val_ratio + test_ratio)))

    #Utworzenie katalogów
    os.makedirs(os.path.join(train_data_folder, class_name), exist_ok=True)
    os.makedirs(os.path.join(validation_data_folder, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_data_folder, class_name), exist_ok=True)

    #Kopiowanie obrazów do odpowiednich katalogów
    for image in train_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(train_data_folder, class_name, image))

    for image in val_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(validation_data_folder, class_name, image))

    for image in test_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(test_data_folder, class_name, image))

print("Dane zostały podzielone na zbiory treningowy, walidacyjny i testowy.")

In [ ]:
#Wyświetlenie liczby zdjęć w każdym zbiorze
train_images_count = sum([len(os.listdir(os.path.join(train_data_folder, class_name))) for class_name in os.listdir(train_data_folder)])
val_images_count = sum([len(os.listdir(os.path.join(validation_data_folder, class_name))) for class_name in os.listdir(validation_data_folder)])
test_images_count = sum([len(os.listdir(os.path.join(test_data_folder, class_name))) for class_name in os.listdir(test_data_folder)])

print("Liczba zdjęć w katalogu treningowym: ", train_images_count)
print("Liczba zdjęć w katalogu walidacyjnym: ", val_images_count)
print("Liczba zdjęć w katalogu testowym: ", test_images_count)

In [ ]:
# Prezentacja liczebności poszczególnych zbiorów w postaci wykresu słupkowego
datasets = ['Zbiór treningowy', 'Zbiór walidacyjny', 'Zbiór testowy']
counts = [train_images_count, val_images_count, test_images_count]
colors = ['white', 'white', 'white']
hatches = ['|', '.', 'x']
plt.figure(figsize=(8, 6))
bars = plt.bar(datasets, counts, color=colors, edgecolor='black', alpha=0.8)

for bar, hatch in zip(bars, hatches):
    bar.set_hatch(hatch)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 10, str(height), ha='center', fontsize=12)

plt.title('Wykres podziału danych', fontsize=16)
plt.xlabel('Zbiór danych', fontsize=12)
plt.ylabel('Ilość zdjęć', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Stworzenie generatorów dla danych treningowych, walidacyjnych i testowych

#Ustalenie reguł wstępnego przetwarzania obrazów
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
     preprocessing_function=preprocess_input,
)

#Utworzenie generatora treningowego i załadowanie danych za pomocą flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
#Utworzenie generatora walidacyjnego i załadowanie danych
validation_generator = test_datagen.flow_from_directory(
    validation_data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
#Utworzenie generatora testowego i załadowanie danych
test_generator = test_datagen.flow_from_directory(
    test_data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Sprawdzenie wymiarów danych
for data_batch, labels_batch in train_generator:
    print(data_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
# Wizualizacja 5 przykładowych obrazów po zmodyfikowaniu
for data_batch, labels_batch in train_generator:
    for i in range(5):  # Wyświetl 5 obrazów
        plt.subplot(1, 5, i + 1)
        plt.imshow(data_batch[i].astype('uint8'))
        plt.axis('off')
    plt.show()
    break

In [ ]:
# Obliczenie wag klas na podstawie liczby przykładów w każdej klasie
class_labels = train_generator.classes
num_classes = len(train_generator.class_indices)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=class_labels
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Wagi klas:", class_weights_dict)

In [ ]:
# Wyświetlenie klas z wagami w posortowanej tabeli
df_class_weights = pd.DataFrame(list(class_weights_dict.items()), columns=["Klasa", "Waga"])
df_class_weights_sorted = df_class_weights.sort_values(by="Waga", ascending=False)

print("Tabela wag klas:")
print(df_class_weights_sorted)

#Załadowanie modelu ResNet50 z wytrenowanymi wagami

In [ ]:
#Załadowanie modelu ResNet50 bez warstwy klasyfikacyjnej i zablokowanie wszystkich jej warstw do dalszego uczenia
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model.summary()

for layer in resnet_model.layers:
    layer.trainable = False

#Utworzenie nowej częsci klasyfikacyjnej

In [ ]:
#Stworzenie nowej części odpowiedzialnej za klasyfikację z warstwą bazową ResNet50
model = Sequential([
    resnet_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(120, activation='softmax')
])

model.summary()

In [ ]:
#Kompilacja modelu z wykorzystaniem niskiego współczynnika uczenia, aby zachować stabilność wcześniej wytrenowanych wag
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss= CategoricalCrossentropy(),
              metrics=['accuracy'])

#Konfiguracja funkcji pozwalających na wcześniejsze zatrzymanie treningu, zapisanie najlepszego modelu oraz automatyczne dostosowanie tempa uczenia
training_helpers = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint('resnet_frozen_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

#Pierwszy etap trenowania modelu

In [ ]:
#Trenowanie modelu z wykorzystaniem generatorów danych,uwzględniając wagi klas oraz callbacki
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=training_helpers,
    class_weight=class_weights_dict
)

In [ ]:
#Ocena modelu na zbiorze testowym po pierwszym etapie trenowania
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Wynik na zbiorze testowym - Strata: {test_loss}, Dokładność: {test_accuracy}")

#Dopracowywanie modelu

In [ ]:
print("Architektura modelu przed odblokowaniem warstw do fine-tuningu:")
resnet_model.summary()

In [ ]:
# Liczba trenowanych parametrów przed odblokowaniem warstw do fine-tuningu
trainable_params_before = sum([tf.keras.backend.count_params(w) for w in resnet_model.trainable_weights])
print(f"Liczba trenowanych parametrów przed odblokowaniem warstw do fine-tuningu: {trainable_params_before}")


Dostosowywaniu wstępnie wytrenowanego modelu do klasyfikacji ras psów

In [ ]:
# Odblokowanie wybranych warstw modelu bazowego po pierwszym etapie treningu w celu przeprowadzenia dostrajania
# Następnie kompilacja modelu z niższym tempem uczenia, aby uchronić wcześniej wyuczone parametry
fine_tune_at = 160

for layer in resnet_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in resnet_model.layers[fine_tune_at:]:
    layer.trainable = True

print(f"Liczba wszystkich części w modelu ResNet50: {len(resnet_model.layers)}")

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
print("Architektura modelu po odblokowaniu 15 ostatnich warstw:")
resnet_model.summary()

In [ ]:
# Liczba trenowanych parametrów po odblokowaniu warstw
trainable_params_after = sum([tf.keras.backend.count_params(w) for w in resnet_model.trainable_weights])
print(f"Liczba trenowanych parametrów po zablokowaniu warstw: {trainable_params_after}")


#Dalsze trenowanie już wcześniej wytrenowanego modelu z odblokowaniem końcowych warstw, aby lepiej dopasować go do rozpoznawania ras psów

In [ ]:
# Dalsze trenowanie modelu przez 20 epok z mniejszym tempem uczenia z wykorzystaniem callbacków do monitorowania procesu
fine_tuning_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint('resnet_finetuned_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

fine_tune_history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=fine_tuning_callbacks,
    class_weight=class_weights_dict  # Ważenie klas
)

#Ewaluacja i ocena modelu na zbiorze testowym

In [ ]:
#Ocena modelu na zbiorze testowym po fine-tuningu
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Wynik na zbiorze testowym po fine-tuningu- Strata: {test_loss}, Dokładność: {test_accuracy}")

In [ ]:
#Ocena modelu na zbiorze walidacyjnym
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Wynik na zbiorze walidacyjnym po fine-tuningu: {val_acc * 100:.2f}%")

In [ ]:
# Wizualizacja dokładności modelu na zbiorze treningowym i walidacyjnym w trakcie treningu
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], linestyle='-', marker='o', color='black', label='Dokładność (trening)')
plt.plot(history.history['val_accuracy'], linestyle='--', marker='s', color='black', label='Dokładność (walidacja)')
plt.xlabel('Epoka', fontsize=12)
plt.ylabel('Dokładność', fontsize=12)
plt.ylim([0, 1])
plt.legend(loc='lower right', fontsize=10)
plt.title('Dokładność treningu i walidacji', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Wizualizacja straty modelu na zbiorze treningowym i walidacyjnym w trakcie treningu
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], linestyle='-', marker='o', color='black', label='Strata (trening)')
plt.plot(history.history['val_loss'], linestyle='--', marker='s', color='black', label='Strata (walidacja)')
plt.xlabel('Epoka', fontsize=12)
plt.ylabel('Strata', fontsize=12)
plt.legend(loc='upper right', fontsize=10)
plt.title('Strata treningu i walidacji', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
#Uzyskanie prognoz modelu na zbiorze testowym i porównanie z rzeczywistymi etykietami
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())


In [ ]:
# Przygotowanie zbioru testowego bez tasowania danych.
#Dane testowe są podawane modelowi w tej samej kolejności, co ułatwia porównanie wyników między różnymi sesjami testowania.
test_generator = test_datagen.flow_from_directory(
    test_data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Pobranie rzeczywistych etykiet i prognoz
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
predicted_classes = np.argmax(predictions, axis=1)

print(f"Liczba prawdziwych etykiet: {len(true_classes)}")
print(f"Liczba przewidywanych etykiet:: {len(predicted_classes)}")

In [ ]:
#Wizualizacja wyników predykcji dla 5 losowych obrazów testowych i porównanie rzeczywistej klasy z przewidywaną przez model
filepaths = test_generator.filepaths

for i in random.sample(range(len(true_classes)), 5):
    img_path = filepaths[i]
    img = plt.imread(img_path)
    plt.imshow(img)
    plt.title(f"Rzeczywista klasa: {class_labels[true_classes[i]]}\n"
              f"Przewidziana klasa: {class_labels[predicted_classes[i]]}")
    plt.axis('off')
    plt.show()

In [ ]:
# Obliczanie dokładności dla każdej klasy na podstawie porównania prawdziwych etykiet z przewidzianymi
def compute_class_accuracies(true_classes, predicted_classes, num_classes):
    correct_per_class = np.zeros(num_classes)
    total_per_class = np.zeros(num_classes)

    for i in range(len(true_classes)):
        total_per_class[true_classes[i]] += 1
        if true_classes[i] == predicted_classes[i]:
            correct_per_class[true_classes[i]] += 1

    return correct_per_class / total_per_class

class_accuracies = compute_class_accuracies(true_classes, predicted_classes, len(class_labels))

In [ ]:
def plot_top_accuracies(class_labels, class_accuracies, top_n=10):
    sorted_indices = np.argsort(class_accuracies)[::-1][:top_n]
    top_labels = [class_labels[i] for i in sorted_indices]
    top_accuracies = class_accuracies[sorted_indices]

    plt.figure(figsize=(12, 6))
    bars = plt.barh(top_labels, top_accuracies, color='white', edgecolor='black', height=0.7)

    for bar in bars:
        bar.set_hatch('/')  # Kropki

    plt.xlabel('Dokładność', fontsize=14, fontweight='bold')
    plt.ylabel('Klasy', fontsize=14, fontweight='bold')
    plt.title(f'Top {top_n} klas o największej dokładności', fontsize=16, fontweight='bold')

    for i, v in enumerate(top_accuracies):
        plt.text(v + 0.02, i, round(v, 2), ha='left', va='center', fontsize=12, fontweight='bold')

    plt.xlim(0, 1)
    plt.show()

plot_top_accuracies(class_labels, class_accuracies)

In [ ]:
def plot_low_accuracies(class_labels, class_accuracies, threshold=0.5):
    low_acc_indices = np.where(class_accuracies < threshold)[0]
    if len(low_acc_indices) == 0:
        print("Brak klas z dokładnością poniżej 50%.")
        return

    low_labels = [class_labels[i] for i in low_acc_indices]
    low_accuracies = class_accuracies[low_acc_indices]

    plt.figure(figsize=(12, 6))
    bars = plt.barh(low_labels, low_accuracies, color='white', edgecolor='black', height=0.7)

    for bar in bars:
        bar.set_hatch('-')  # Kreski poziome

    plt.xlabel('Dokładność', fontsize=14, fontweight='bold')
    plt.ylabel('Klasy', fontsize=14, fontweight='bold')
    plt.title('Klasy z dokładnością poniżej 50%', fontsize=16, fontweight='bold')

    for i, v in enumerate(low_accuracies):
        plt.text(v + 0.02, i, round(v, 2), ha='left', va='center', fontsize=12, fontweight='bold')

    plt.xlim(0, 1)
    plt.show()

plot_low_accuracies(class_labels, class_accuracies)

In [ ]:
# Wizualizacja obrazów dla klasy
def plot_images_for_class(class_name, image_directory, title, num_images=9):
    plt.figure(figsize=(10, 10))
    images = os.listdir(os.path.join(image_directory, class_name))
    for i in range(min(num_images, len(images))):
        img_name = random.choice(images)
        img_path = os.path.join(image_directory, class_name, img_name)
        img = Image.open(img_path)

        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.title(class_name, fontsize=8)
        plt.axis('off')
    plt.suptitle(title, fontsize=15)
    plt.tight_layout()
    plt.show()

#Wyświetlenie kla z dokładnością < 50% i wizualizacja obrazów z tych klas
low_accuracy_classes = [class_labels[i] for i, acc in enumerate(class_accuracies) if acc < 0.5]
print("Klasy z dokładnością mniejszą niż 50%:")
print(", ".join(low_accuracy_classes))

for cls in low_accuracy_classes:
    plot_images_for_class(cls, test_data_folder, f"Niska dokładność - klasa: {cls}")

In [ ]:
# Wyświetlanie najgorzej przewidywanej klasy i klasy z którą jest najczęściej mylona
def display_confusion_details(true_classes, predicted_classes, class_labels):
    cm = confusion_matrix(true_classes, predicted_classes)
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    worst_class_idx = np.argmin(class_accuracies)
    worst_class_name = class_labels[worst_class_idx]
    worst_class_accuracy = class_accuracies[worst_class_idx]

    cm_without_diagonal = cm[worst_class_idx].copy()
    cm_without_diagonal[worst_class_idx] = 0

    # Znajdowanie klasy najczęściej mylonej z najgorzej przewidywaną klasą
    most_confused_class_idx = np.argmax(cm_without_diagonal)
    most_confused_class_name = class_labels[most_confused_class_idx]
    most_confused_value = cm[worst_class_idx, most_confused_class_idx]

    print(f"Najgorzej przewidywana klasa: {worst_class_name} (dokładność: {worst_class_accuracy:.2f})")
    print(f"Klasa najczęściej mylona z '{worst_class_name}': {most_confused_class_name} "
          f"({most_confused_value} błędnych predykcji)\n")

    return worst_class_name, most_confused_class_name

# Wywołanie funkcji
worst_class, most_confused_class = display_confusion_details(true_classes, predicted_classes, class_labels)

In [ ]:
# Wizualizacja obrazów dla klasy najgorzej przewidywanej
plot_images_for_class(worst_class, test_data_folder, f"Najgorzej przewidywana klasa: {worst_class}")

In [ ]:
#Wizualizacja klasy z którą najgorzej przewidywana klasa jest najczęściej mylona
plot_images_for_class(most_confused_class, test_data_folder, f"Klasa z która jest najczesciej mylona : {most_confused_class}")

In [ ]:
# Obliczenie liczby próbek w każdej klasie i wyświetlenie 10 klas z najmniejszą liczbą próbek
class_counts = {cls: len(os.listdir(os.path.join(image_directory, cls))) for cls in class_labels}
sorted_class_counts = sorted(class_counts.items(), key=lambda x: x[1])

min_classes = sorted_class_counts[:10]
min_class_labels = [cls for cls, _ in min_classes]
min_class_counts = [count for _, count in min_classes]


plt.figure(figsize=(10, 6))
bars = plt.bar(min_class_labels, min_class_counts, color='white', edgecolor='black', hatch='/')

for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(int(bar.get_height())),
             ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.xlabel('Klasy', fontsize=12)
plt.ylabel('Liczba próbek', fontsize=12)
plt.title('10 klas z najmniejszą liczbą próbek', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Wyświetlenie raportu klasyfikacji

In [ ]:
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [ ]:
report = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)

df_report = pd.DataFrame(report).T
df_report = df_report.iloc[:-3]
df_sorted = df_report.sort_values(by='f1-score', ascending=False)

best_classes = df_sorted.head(5)
worst_classes = df_sorted.tail(5)

df_display = pd.concat([best_classes, worst_classes])
df_display = df_display[['precision', 'recall', 'f1-score', 'support']]

print("Top 5 najlepszych i najgorszych klas według F1-score\n")
print(df_display)

# Wizualizacja mapy cieplnej
plt.figure(figsize=(8, 6))
sns.heatmap(df_display.iloc[:, :-1], annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, cbar=True)

plt.title('Mapa cieplna dokładności dla wybranych klas', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, rotation=0)
plt.show()

**Predykcja klas dla nieznanych obrazów**

In [ ]:
img_paths = ["pudel.jpg", "beagle.jpg", "yorkbeagle.jpg", "Owczarek-niemiecki-kolorowanka.jpg", "pies-bernenski.jpg", "człowiekpies.jpg"]

# Pętla przez obrazy
for img_path in img_paths:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predykcja klasy przez model
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.title(f"Przewidziana klasa: {predicted_label}")
    plt.axis('off')
    plt.show()
    print(f"Obraz: {img_path}")
    print(f"Przewidziana klasa: {predicted_label}\n")

In [ ]:
model.save('model25.12.keras')